problem a:\
EM algorithm with K: 2\
pi =  [0.40966009 0.59033991]\
mu =  [[ 0.03139044  3.0670068 ][ 3.0209811  -0.17917812]]\
sigma =  [[[ 1.02201118 -0.06296114] [-0.06296114  0.95982257]] 
         [[ 1.00252974  0.16301188] [ 0.16301188  0.93963276]]]\

EM algorithm with K: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 3.17197741 -2.27654432][-2.27654432  3.49632749]]]\

Two-step EM algorithm with k: 2\
pi =  [0.40712577 0.59287423]\
mu =  [[ 0.01327772  3.07239955][ 3.02063968 -0.16900507]]\
sigma =  [[[ 1.1080443  -0.12730306][-0.12730306  1.02665851]]\
         [[ 1.0947716   0.13547323][ 0.13547323  0.96215989]]]\

Two-step EM algorithm with k: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 6.47775003 -5.82548776][-5.82548776  7.30632925]]]\

problem b:\
EM algorithm with K: 2\
pi =  [0.40966009 0.59033991]\
mu =  [[ 0.03139044  3.0670068 ][ 3.0209811  -0.17917812]]\
sigma =  [[[ 1.02201118 -0.06296114][-0.06296114  0.95982257]]\
         [[ 1.00252974  0.16301188][ 0.16301188  0.93963276]]]\

EM algorithm with K: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 3.17197741 -2.27654432][-2.27654432  3.49632749]]]\

Two-step EM algorithm with k: 2\
pi =  [0.43220299 0.56779701]\
mu =  [[ 0.13372122  2.94555262][ 3.06178159 -0.21560942]]\
sigma =  [[[ 1.25039876 -0.31855648][-0.31855648  1.28499556]]\
         [[ 1.12192566  0.30353082][ 0.30353082  0.99064177]]]\

Two-step EM algorithm with k: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 3.74079773 -3.25880804][-3.25880804  5.19254325]]]\

problem c\
EM algorithm with K: 2\
pi =  [0.40966009 0.59033991]\
mu =  [[ 0.03139044  3.0670068 ][ 3.0209811  -0.17917812]]\
sigma =  [[[ 1.02201118 -0.06296114][-0.06296114  0.95982257]]
         [[ 1.00252974  0.16301188][ 0.16301188  0.93963276]]]\
EM algorithm with K: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 3.17197741 -2.27654432][-2.27654432  3.49632749]]]\

Two-step EM algorithm with k: 2\
pi =  [0.58931341 0.41068659]\
mu =  [[ 2.98483881 -0.1692579 ][ 0.09072516  3.04465801]]\
sigma =  [[[ 1.32956201e+00  6.61719672e-04][ 6.61719672e-04  1.01299268e+00]]\
         [[ 1.22808966e+00 -1.94985891e-01][-1.94985891e-01  1.02294942e+00]]]\
         
Two-step EM algorithm with k: 1\
pi =  [1.]\
mu =  [[1.79626513 1.15065428]]\
sigma =  [[[ 3.36348805 -2.44457917][-2.44457917  3.64376426]]]\

In [1]:
from scipy.stats import multivariate_normal as mn
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [2]:
# define Expectation Step
def E_step(x, pi, mu, sigma, K): 
    n = x.shape[0]
    E_y = np.zeros((n,K))
    E_yi = np.zeros(K)

    for i in range(n):
        for k in range(K):
            E_yi[k] = mn.pdf(x[i,:], mean = mu[k,:], cov = sigma[k]) * pi[k]
        E_yi_sum = np.sum(E_yi)
        E_y[i,:] = E_yi / E_yi_sum
    return E_y

In [3]:
# define Maximization Step
def M_step(x, mu, E_y, K): 
    n = x.shape[0]
    pi_ = np.sum(E_y, axis = 0) / n # (K,1)
    mu_ = np.dot(E_y.T, x) / np.sum(E_y, axis = 0).reshape(K,1)
    sigma_ = np.zeros((K,2,2))
    for k in range(K):
        sigma_[k] = np.dot( (x - mu[k,:]).T, E_y[:,k].reshape(n,1) * (x - mu[k,:]) ) / np.sum(E_y[:,k])
    return pi_, mu_, sigma_

In [4]:
# EM algorithm with KMeans  #
def EM(x, K):
    #  KMeans with clusters = K#
    k_means = KMeans(n_clusters = K, init = 'k-means++').fit(x)
    label = k_means.labels_
    center = k_means.cluster_centers_
    # Initialize  #
    pi = [0.5, 0.5]
    mu = center # mean centers
    sigma = np.zeros((K,2,2))
    for k in range(K):
        sigma[k] = np.dot( (x[label == k] - mu[k,:]).T, (x[label == k] - mu[k,:]) ) / (x[label == k].shape[0] - 1)
   
    # EM calculate #
    delta = np.ones(3)
    while (min(delta) > 1e-3):
        E_y = E_step(x, pi, mu, sigma, K)
        pi_, mu_, sigma_ = M_step(x, mu, E_y, K)
        delta[0] = np.linalg.norm(pi - pi_)
        delta[1] = np.linalg.norm(mu - mu_)
        delta[2] = np.linalg.norm(sigma - sigma_)
        # Updata  #
        pi = pi_
        mu = mu_
        sigma = sigma_
    print("........................")
    print(' EM algorithm with K:',K)
    print('pi = ', pi)
    print('mu = ', mu)
    print('sigma = ', sigma)

In [79]:
def prov_EM(x, l, K):
#     initialize
    N = x.shape[0]
    seed = np.random.randint(0, N, size = l, dtype = int) 
    mu = x[seed,:]
    pi = np.ones(4) / l
    sigma = np.zeros((l,2,2)) 
    for i in range(l):
        mu_j = np.delete(mu, i, axis=0) 
        mu_i = np.tile(mu[i], (l-1, 1)) 
        sigma[i] = min(np.linalg.norm(mu_i-mu_j, axis = 1)) * np.identity(2)
#   first em step
    E_y = E_step(x, pi, mu, sigma, l) 
    pi, mu, sigma = M_step(x, mu, E_y, l) # M step
# pruning step
    id_keep = (pi > 1/(4*l)) 
    pi_ = pi[id_keep]
    mu_ = mu[id_keep]
    sigma_ = sigma[id_keep]

    r = pi_.shape[0] 
    S = []
    m = np.random.randint(0, r, size = 1, dtype = int) 
    S = np.append(S,mu_[m]).reshape(1,2)

    mu_j = np.tile(mu_[m], (r, 1))
    dis = np.linalg.norm(mu_ - mu_j, axis = 1)    
    i = np.argmax(dis) 
    S = np.append(S,mu_[i].reshape(1,2), axis = 0) 
# keep center cluster
    pi = pi_[[m[0],i]]
    mu = mu_[[m[0],i]]
    sigma = sigma_[[m[0],i]]
#  Second Em step 
    E_y = E_step(x, pi, mu, sigma, K) 
    pi, mu, sigma = M_step(x, mu, E_y, K) 
    E_y = E_step(x, pi, mu, sigma, K) 
    
    print("...............................")
    print('Two-step EM algorithm with k:',K)
    print('pi = ', pi)
    print('mu = ', mu)
    print('sigma = ', sigma)

In [88]:
# a xeasy #
xeasy = np.loadtxt('D:/2020fall/5635 machine learning/5635 hw/hw9/xeasy.txt', delimiter = ',')
EM_KMeans(xeasy, 2)
EM_KMeans(xeasy, 1)
prov_EM(xeasy, 4, 2)
prov_EM(xeasy, 3, 1)

........................
 EM algorithm with K: 2
pi =  [0.59033991 0.40966009]
mu =  [[ 3.0209811  -0.17917812]
 [ 0.03139044  3.0670068 ]]
sigma =  [[[ 1.00252974  0.16301188]
  [ 0.16301188  0.93963276]]

 [[ 1.02201118 -0.06296114]
  [-0.06296114  0.95982257]]]
........................
 EM algorithm with K: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 3.17197741 -2.27654432]
  [-2.27654432  3.49632749]]]
...............................
Two-step EM algorithm with k: 2
pi =  [0.57347626 0.42652374]
mu =  [[ 3.06750072 -0.20339836]
 [ 0.08704388  2.97122598]]
sigma =  [[[ 0.9363815   0.19640278]
  [ 0.19640278  0.94933851]]

 [[ 1.10389949 -0.18412177]
  [-0.18412177  1.18671259]]]
...............................
Two-step EM algorithm with k: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 6.73023261 -6.66448747]
  [-6.66448747  8.90741914]]]


In [90]:
# b x1 #
x1 = np.loadtxt('D:/2020fall/5635 machine learning/5635 hw/hw9/x1.txt', delimiter = ',')
EM(xeasy, 2)
EM(xeasy, 1)
prov_EM(xeasy, 4, 2)
prov_EM(xeasy, 3, 1)

........................
 EM algorithm with K: 2
pi =  [0.59033991 0.40966009]
mu =  [[ 3.0209811  -0.17917812]
 [ 0.03139044  3.0670068 ]]
sigma =  [[[ 1.00252974  0.16301188]
  [ 0.16301188  0.93963276]]

 [[ 1.02201118 -0.06296114]
  [-0.06296114  0.95982257]]]
........................
 EM algorithm with K: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 3.17197741 -2.27654432]
  [-2.27654432  3.49632749]]]
...............................
Two-step EM algorithm with k: 2
pi =  [0.37653396 0.62346604]
mu =  [[ 2.97589748 -0.5420279 ]
 [ 1.08384199  2.17292698]]
sigma =  [[[ 0.99650089  0.23471527]
  [ 0.23471527  0.71599905]]

 [[ 3.36368775 -1.98673808]
  [-1.98673808  2.52308487]]]
...............................
Two-step EM algorithm with k: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 6.39301188 -5.89430203]
  [-5.89430203  7.55967142]]]


In [91]:
# c x2 #
x2 = np.loadtxt('D:/2020fall/5635 machine learning/5635 hw/hw9/x2.txt', delimiter = ',')
EM(xeasy, 2)
EM(xeasy, 1)
prov_EM(xeasy, 4, 2)
prov_EM(xeasy, 3, 1)

........................
 EM algorithm with K: 2
pi =  [0.59033991 0.40966009]
mu =  [[ 3.0209811  -0.17917812]
 [ 0.03139044  3.0670068 ]]
sigma =  [[[ 1.00252974  0.16301188]
  [ 0.16301188  0.93963276]]

 [[ 1.02201118 -0.06296114]
  [-0.06296114  0.95982257]]]
........................
 EM algorithm with K: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 3.17197741 -2.27654432]
  [-2.27654432  3.49632749]]]
...............................
Two-step EM algorithm with k: 2
pi =  [0.26866385 0.73133615]
mu =  [[-0.29923977  3.27495535]
 [ 2.56607038  0.37027046]]
sigma =  [[[ 0.83284403  0.1494904 ]
  [ 0.1494904   0.71374244]]

 [[ 1.94412611 -1.05998521]
  [-1.05998521  2.40759936]]]
...............................
Two-step EM algorithm with k: 1
pi =  [1.]
mu =  [[1.79626513 1.15065428]]
sigma =  [[[ 5.14141837 -4.15966378]
  [-4.15966378  5.29690896]]]
